# MongoDB Atlas Search: Language Analyziers

## Setup

In [2]:
import sys
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pprint

# Load environment variables from a .env file, overriding any existing variables in the environment.
# This is useful for not cluttering the code with sensitive information like API keys and database URIs.
load_dotenv(override=True)

MONGO_URI = os.environ["MONGO_URI"]

client = MongoClient(MONGO_URI)['PIA_LOCAL']

pp = pprint.PrettyPrinter(indent=1)

#### Atlas index definition

```json
{
  "mappings": {
    "fields": {
      "TRANSLATIONS": {
        "dynamic": false,
        "fields" : {
          "LOCALE_CODE" : {
            "type" : "string"
          },
          "TEXT_VALUE" : {
            "analyzer" : "lucene.standard",
            "multi" : {
              "englishAnalyzer": {
                "analyzer": "lucene.english",
                "searchAnalyzer": "lucene.english",
                "type": "string"
              },
              "spanishAnalyzer": {
                "analyzer": "lucene.spanish",
                "searchAnalyzer": "lucene.spanish",
                "type": "string"
              }
            }
          }
        },
        "type": "embeddedDocuments"
      }
    }
  }
}
```


### Embedded document search using English Analyzer

In [3]:
pipeline = [
    {
        '$search': {
            'index': 'embedded', 
            'embeddedDocument': {
                'path': 'TRANSLATIONS', 
                'operator': {
                    'text': {
                        'query': 'Botellero',
                        'path': {'value' : 'TRANSLATIONS.TEXT_VALUE',
                                 'multi' : 'spanishAnalyzer'},
                    }
                }
            }
        }
    },
    { '$limit': 2 }
]

result = list(client['TRANSLATIONS'].aggregate(pipeline))
pp.pprint(result)

[{'TEXT_ID': '0027809',
  'TRANSLATIONS': [{'CREATED_DATE': datetime.datetime(2003, 3, 25, 13, 33, 58),
                    'DELETED_DATE': datetime.datetime(1970, 1, 1, 0, 0),
                    'LOCALE_CODE': 'en-ZZ',
                    'OBJECT_TYPE': 'BEN',
                    'TEXT_PART': 'BEN-001',
                    'TEXT_STATUS': 'source',
                    'TEXT_VALUE': 'Reversible, so you can choose the placement '
                                  'of drawers and bottle rack.',
                    'UPDATED_DATE': datetime.datetime(2013, 6, 14, 14, 16, 37),
                    'USER_ID': '',
                    'VERSION_NO': '8'},
                   {'CREATED_DATE': datetime.datetime(2020, 4, 2, 18, 38, 1),
                    'DELETED_DATE': datetime.datetime(1970, 1, 1, 0, 0),
                    'LOCALE_CODE': 'es-CL',
                    'OBJECT_TYPE': 'BEN',
                    'TEXT_PART': 'BEN-001',
                    'TEXT_STATUS': 'Translated',
                 